<a href="https://colab.research.google.com/github/pradeepsai7/OIBSIP/blob/main/Project_3_Proposal_Level_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DATASET-1

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import IsolationForest
import zipfile

# Unzip the archive file if it exists
zip_file_path = "/content/archive (10).zip"
if os.path.exists(zip_file_path):
    print(f"Unzipping {zip_file_path}...")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(".") # Extract to current directory
    print("Unzipping complete.")
else:
    print(f"Warning: Zip file {zip_file_path} not found. Please ensure the data is available.")

# Now, try to find the CSV file
csv_files = [f for f in os.listdir() if f.endswith(".csv")]
if not csv_files:
    raise FileNotFoundError("No CSV files found after unzipping. Please check the archive content.")

file = csv_files[0]
df = pd.read_csv(file)

df.columns = df.columns.str.lower().str.strip()

label_col = [c for c in df.columns if c in ["class", "fraud", "is_fraud"]][0]

num_cols = df.select_dtypes(include=["int64", "float64"]).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

X = df.drop(label_col, axis=1)
y = df[label_col]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

iso = IsolationForest(contamination=0.05, random_state=42)
df["anomaly"] = iso.fit_predict(X_scaled)
df["anomaly"] = df["anomaly"].map({1: 0, -1: 1})

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

df["fraud_prediction"] = model.predict(X_scaled)

stream_data = X_scaled[:20]
df.loc[:19, "realtime_prediction"] = model.predict(stream_data)

df.to_csv("fraud_detection_output.csv", index=False)


Unzipping /content/archive (10).zip...
Unzipping complete.
